# Online model definition and data preparation

## Model definition

In [ ]:
from creme import compose
from creme import linear_model
from creme import preprocessing


def parse(row):
    import datetime as dt
    row['date'] = dt.datetime.fromisoformat(row['date']).timestamp()
    return row

model = compose.FuncTransformer(parse) \
    | compose.Select('date', 'new_deceased', 'new_recovered',
                     'cumulative_confirmed', 'cumulative_deceased',
                     'cumulative_recovered') \
    | preprocessing.StandardScaler() \
    | linear_model.LinearRegression()


### Upload model to chantilly

In [ ]:
import dill
import requests

requests.post('http://localhost:5000/api/model', data=dill.dumps(model))

## Download data

In [ ]:
!wget https://storage.googleapis.com/covid19-open-data/v3/epidemiology.csv

In [ ]:
!wget https://storage.googleapis.com/covid19-open-data/v3/vaccinations.csv

## Data preparation

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('epidemiology.csv')

In [ ]:
data.head()

### Filter data by state DE_HB (Bremen)

In [ ]:
bremen_data = data[data.location_key.apply(lambda x: "DE_HB" == x if isinstance(x, str) else False)]

### Drop na columns with no data

In [ ]:
bremen_data = bremen_data.drop(columns=["new_tested", "cumulative_tested"])

### Bremen data insights

In [ ]:
bremen_data.head()

In [ ]:
max(bremen_data.new_confirmed)

In [ ]:
bremen_data.to_csv("bremen_epidemiology.csv")

## Vaccination data

In [ ]:
vacc_data = pd.read_csv('vaccinations.csv')

In [ ]:
merged_data = pd.merge(data, vacc_data, on=['date', 'location_key'])

In [ ]:
bremen_vacc_data = merged_data[merged_data.location_key.apply(lambda x: "DE_HB" == x if isinstance(x, str) else False)]

In [ ]:
bremen_vacc_data = bremen_vacc_data[['date', 'location_key', 'new_confirmed', 'new_deceased',
       'new_recovered', 'cumulative_confirmed', 'cumulative_deceased',
       'cumulative_recovered', 'new_persons_vaccinated',
       'cumulative_persons_vaccinated', 'new_persons_fully_vaccinated',
       'cumulative_persons_fully_vaccinated', 'new_vaccine_doses_administered',
       'cumulative_vaccine_doses_administered']]

In [ ]:
bremen_vacc_data